In [6]:
import ast
import numpy as np
import pandas as pd

from opportunistic_planning import prediction, processing

In [11]:
data = pd.read_csv('all_task_environments_list_2022-09-12_unique_sequences.csv', header=0)

In [19]:
distances_dict = processing.generate_distances_dict(data, use_string_for_seq=False,
                                                   dimensions=[[2, 'xy'], [3, 'xyz']])

In [25]:
distances_dict['xy']

{'a1': {(-0.451354, -0.413918): {'plate': 1.4459178173547762,
   'plate_small': 1.368096623795264,
   'cup': 1.3392278666492867,
   'glass': 1.6531706987885433,
   'knife': 1.2219225591673148,
   'spoon_small': 1.1675352349787993},
  (0.513, -0.531): {'plate': 1.571425690218917,
   'plate_small': 1.6173697783747538,
   'cup': 1.786728176343005,
   'glass': 2.08311358691767,
   'knife': 1.3764788073116128,
   'spoon_small': 1.695938442180317}},
 'a3': {(-0.451354, -0.413918): {'cup': 1.35337885269314,
   'glass': 1.6531706987885433,
   'wineglass': 1.7879633892415134,
   'plate': 1.4459178173547762,
   'fork': 1.183447393772955,
   'knife': 1.2219225591673148,
   'spoon': 1.2727831868550903},
  (0.513, -0.531): {'cup': 1.8683849247307152,
   'glass': 2.08311358691767,
   'wineglass': 2.015545721357866,
   'plate': 1.571425690218917,
   'fork': 1.4649661266432066,
   'knife': 1.3764788073116128,
   'spoon': 1.3513424160829852}},
 'a5': {(-0.451354, -0.413918): {'knife': 1.221922559167314

### predict sequences

In [20]:
results = pd.DataFrame()

for row in range(0, len(data)):
        use_string_for_seq = False
        coordinates = {key: ast.literal_eval(value) for key, value in
                       (elem.split(': ') for elem in data.at[row, 'coordinates'].split(';'))}

        start_coordinates = list(ast.literal_eval(data.at[row, 'start_coordinates']))
        ID = str(data.at[row,'ID'])
        
        if use_string_for_seq == True:
            seq = str(data.at[row, 'sequence'])
            objects = list(data.at[row, 'sequence'])
        else:
            seq = [elem for elem in data.at[row, 'sequence'].split(',')]
            objects = [elem for elem in data.at[row, 'sequence'].split(',')]

        # get list of objects that have relational dependencies, if any (else set to empty list)
        try:
            strong_k = list(data.at[row, 'strong_k'].split(','))
        except AttributeError:
            strong_k = []

        try:
            mid_k = list(data.at[row, 'mid_k'].split(','))
        except AttributeError:
            mid_k = []

        try:
            food_k = list(data.at[row, 'food_k'].split(','))
        except AttributeError:
            food_k = []
            
        k1 = {obj: 1.7 if obj in food_k else 0.2 if obj in strong_k \
              else 0.3 if obj in mid_k else 1.0 for obj in objects}
        try:
            c1 = {obj: 1.8 if obj in data.at[row, 'containment'] else 1.0 for obj in objects}
        except:
            c1 = {obj: 1.0 for obj in objects}
            
        dim = [2, 'xy']
        
        predicted = prediction.predict_prequential(distances_dict, ID, objects, coordinates,
                                                  start_coordinates, c1, k1, dim)
        
        results.at[row, 'predicted'] = predicted
        results.at[row, 'observed'] = seq
        results.at[row, 'ID'] = ID

IndexError: list index out of range

In [10]:
results

,median,median_normalized,ID
0,3.0,0.500000,k01
1,2.0,0.400000,k04
2,1.0,0.200000,k06
3,6.0,0.750000,k08
4,3.0,0.333333,k11
5,3.0,0.375000,k12
6,4.0,0.400000,k14
7,5.0,0.555556,k16
8,5.0,0.625000,k17
9,4.0,0.666667,k18


In [11]:
np.median(results['median'])

4.0

In [12]:
results.to_csv('results/results_cleaning_2D_n100_fitted_to_ts_2023-01-20.csv', header=True, index=False)